# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [76]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    dirs[:] = [d for d in dirs if d not in ['.ipynb_checkpoints']] 
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# set up processed events folder if it doesn't already exist
if not os.path.exists('processed_events'):
    os.makedirs('processed_events')

i = 0

# for every filepath in the file path list 
for raw_file in file_path_list:
    # initiating an empty list of rows that will be generated from each file
    full_data_rows_list = []
    
    # reading csv file 
    with open(raw_file, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
    # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

    with open(f'processed_events/event_datafile_{i}.csv', 'w', encoding = 'utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
        for row in full_data_rows_list:
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
    
    i += 1

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# Connect to a Cassandra instance on local machine 

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create and set Keyspace

In [9]:
query = "CREATE KEYSPACE IF NOT EXISTS sparkifydb "
query = query + "WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}"

session.execute(query)
session.execute("USE sparkifydb")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [54]:
drop_query1 = "DROP TABLE IF EXISTS song_in_session_details;"
drop_query2 = "DROP TABLE IF EXISTS user_session_song_details;"
drop_query3 = "DROP TABLE IF EXISTS song_listens;"

create_query1 = "CREATE TABLE IF NOT EXISTS song_in_session_details "
create_query1 = create_query1 + "(sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY(sessionId, itemInSession));"

create_query2 = "CREATE TABLE IF NOT EXISTS user_session_song_details "
create_query2 = create_query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY(userId, sessionId, itemInSession));"

create_query3 = "CREATE TABLE IF NOT EXISTS song_listens "
create_query3 = create_query3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId));"

session.execute(drop_query1)
session.execute(drop_query2)
session.execute(drop_query3)
session.execute(create_query1)
session.execute(create_query2)
session.execute(create_query3)

In [56]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/processed_events'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

for file in file_path_list:
    print(file)
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
    ## TO-DO: Assign the INSERT statements into the `query` variable
            query = "INSERT INTO song_in_session_details (sessionId, itemInSession, artist, song, length) "
            query = query + "VALUES (%s, %s, %s, %s, %s)"
            ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
            ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
            session.execute(query, [int(line[8]), int(line[3]), line[0], line[9], float(line[5])])
            
            query = "INSERT INTO user_session_song_details (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
            query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
            ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
            ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
            session.execute(query, [int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]])
            
            query = "INSERT INTO song_listens (song, userId, firstName, lastName) "
            query = query + "VALUES (%s, %s, %s, %s)"
            ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
            ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
            session.execute(query, [line[9], int(line[10]), line[1], line[4]])

/home/workspace/processed_events/event_datafile_22.csv
/home/workspace/processed_events/event_datafile_11.csv
/home/workspace/processed_events/event_datafile_0.csv
/home/workspace/processed_events/event_datafile_19.csv
/home/workspace/processed_events/event_datafile_6.csv
/home/workspace/processed_events/event_datafile_1.csv
/home/workspace/processed_events/event_datafile_4.csv
/home/workspace/processed_events/event_datafile_14.csv
/home/workspace/processed_events/event_datafile_5.csv
/home/workspace/processed_events/event_datafile_16.csv
/home/workspace/processed_events/event_datafile_3.csv
/home/workspace/processed_events/event_datafile_13.csv
/home/workspace/processed_events/event_datafile_10.csv
/home/workspace/processed_events/event_datafile_15.csv
/home/workspace/processed_events/event_datafile_28.csv
/home/workspace/processed_events/event_datafile_17.csv
/home/workspace/processed_events/event_datafile_20.csv
/home/workspace/processed_events/event_datafile_25.csv
/home/workspace/

#### Do a SELECT to verify that the data have been inserted into each table

In [60]:
pd.DataFrame(list(session.execute("SELECT artist, song, length FROM song_in_session_details WHERE sessionId = 338 AND itemInSession = 4")))

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


In [61]:
pd.DataFrame(list(session.execute("SELECT artist, song, firstName, lastName FROM user_session_song_details WHERE userId = 10 AND sessionId = 182")))

InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table user_session_song_details"

In [50]:
pd.DataFrame(list(session.execute("SELECT firstName, lastName FROM song_listens WHERE song = 'All Hands Against His Own'"))

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [51]:
query = "DROP TABLE IF EXISTS song_in_session_details;"
query1 = "DROP TABLE IF EXISTS user_session_song_details;"
query2 = "DROP TABLE IF EXISTS song_listens;"

session.execute(query)
session.execute(query1)
session.execute(query2)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()